<a href="https://colab.research.google.com/github/coldjoke123/python1/blob/master/HW_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import np_utils

## 載入資料並且重新調整資料的樣子

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)
y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)

## 對資料做正規化，將資料調整至 $[0,1]$ 區間

降低資料受到極值的影響

In [0]:
x_train_reg = (x_train - x_train.min())/(x_train.max() - x_train.min())
x_test_reg = (x_test - x_test.min())/(x_test.max() - x_test.min())

## 首先，我們增加神經元的個數至每層100個，其餘參數不變

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

In [75]:
model_test1 = Sequential()
model_test1.add(Dense(100, input_dim = 784))
model_test1.add(Activation('sigmoid'))
model_test1.add(Dense(100))
model_test1.add(Activation('sigmoid'))
model_test1.add(Dense(10))
model_test1.add(Activation('softmax'))
model_test1.compile(loss = 'mse', optimizer=SGD(lr = 0.087), metrics=['accuracy'])
model_test1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2533 (Dense)           (None, 100)               78500     
_________________________________________________________________
activation_2533 (Activation) (None, 100)               0         
_________________________________________________________________
dense_2534 (Dense)           (None, 100)               10100     
_________________________________________________________________
activation_2534 (Activation) (None, 100)               0         
_________________________________________________________________
dense_2535 (Dense)           (None, 10)                1010      
_________________________________________________________________
activation_2535 (Activation) (None, 10)                0         
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [76]:
model_test1_20 = model_test1.fit(x_train_reg, y_train, 
                         batch_size=100, 
                         epochs=20,
                         verbose=1,
                         validation_data=(x_test_reg, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 29s 490us/step - loss: 0.0902 - acc: 0.1039 - val_loss: 0.0899 - val_acc: 0.1142
Epoch 2/20
60000/60000 [==============================] - 7s 116us/step - loss: 0.0898 - acc: 0.1131 - val_loss: 0.0897 - val_acc: 0.1137
Epoch 3/20
60000/60000 [==============================] - 7s 123us/step - loss: 0.0896 - acc: 0.1143 - val_loss: 0.0895 - val_acc: 0.1158
Epoch 4/20
60000/60000 [==============================] - 7s 121us/step - loss: 0.0894 - acc: 0.1239 - val_loss: 0.0893 - val_acc: 0.1416
Epoch 5/20
60000/60000 [==============================] - 7s 121us/step - loss: 0.0892 - acc: 0.1467 - val_loss: 0.0891 - val_acc: 0.1456
Epoch 6/20
60000/60000 [==============================] - 8s 126us/step - loss: 0.0890 - acc: 0.1691 - val_loss: 0.0888 - val_acc: 0.1756
Epoch 7/20
60000/60000 [==============================] - 7s 119us/step - loss: 0.0887 - acc: 0.1943 - val_loss: 0.0886 -

In [77]:
score = model_test1.evaluate(x_test_reg, y_test)
print('loss:', score[0])
print('accuracy:', score[1])

10000/10000 [==============================] - 2s 236us/step
loss: 0.07528994209766388
accuracy: 0.4022


## 相較於老師上課的模型，可以發現增加神經元數是能增加準確率的！

## 於是，在Hidden layer有兩層activation function是sigmoid，loss function是mse，optimizer是SGD的條件下，測試神經元的個數以50個為單位從100到800來尋找準確率最高的組合。

In [0]:
def seek_opt(n):
    seek_accu = np.zeros((2*(n-1),n-1))
    for i in range(1,n):
        for j in range(1,n):
            model = Sequential()
            model.add(Dense(50*(i+1), input_dim = 784))
            model.add(Activation('sigmoid'))
            model.add(Dense(50*(j+1)))
            model.add(Activation('sigmoid'))
            model.add(Dense(10))
            model.add(Activation('softmax'))
            model.compile(loss = 'mse', optimizer=SGD(lr = 0.087), metrics=['accuracy'])
            score = model.evaluate(x_test_reg, y_test)
            seek_accu[i-1,j-1] = score[1]
            seek_accu[i-1+(n-1),j-1] = score[0]
    return seek_accu

In [0]:
accu = seek_opt(16)[:16,:]
loss = seek_opt(16)[16:,:]

10000/10000 [==============================] - 26s 3ms/step


In [0]:
opt_acc_value = accu.max()
opt_acc_index = accu.argmax()